# Access Metrics Made Easy: Nearest Destination and Access Count
This notebook provides a simple workflow and toolkit to calculate find access metrics. Primarily, this notebook calculates the nearest destination and the number of locations within a given threshold. The basic workflow is as follows:
1. Import data: Point data for destinations, origin geographies, and a transit cost matrix of pre-computed travel costs (eg. Minutes, Miles, etc.)
2. Spatially join destinations to origins: Based on the geospatial location, this associates each destination with an origin geography. Given that the travel cost between each origin geography is known, we can easily calculate the distance between.
3. Calculate metrics: For the nearest location, we'll simply sort the list of origins and destinations by travel time, then take the first entry. For the count within a given threshold, we can filter the list of origins and destinations by the travel time, and count the number of entries under a given threshold.

---

Getting start: Imports and a helper function for later. Here, we'll install the needed libraries (on the Colab remote machine) and import the libraries:

In [1]:
!pip install pandas geopandas access rtree pygeos access pyarrow

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
# from access import Access, weights, Datasets

def dfToGdf(df, lon, lat, crs='EPSG:4326'):
  '''
    df: pandas dataframe
    lon: longitude column name
    lat: latitude column name
    crs: EPSG code or similar coordinate reference system
  '''
  return gpd.GeoDataFrame(
    df.drop([lon, lat], axis=1),
    crs=crs,
    geometry=[Point(xy) for xy in zip(df[lon], df[lat])])

/Users/susan/opt/miniconda3/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### Public data:
We've provided a set of public datasets to help get you started. Specifically, we have travel cost matrices for US Zip codes and US Census Tracts, with the travel cost as a value in minutes. The data here is in the Apache Parquet format, an efficient format for storing tabular data.

We've also included some base geographies and population data (should you need it) and some sample destination data of Federally Qualified Health Clinics (FQHC's).

In [7]:
matrices = {
    'tract': {
        'car':'https://uchicago.box.com/shared/static/hkipez75z2p7zivtjdgsfzut4rhm6t6h.parquet',
        'bike':'https://uchicago.box.com/shared/static/cvkq3dytr6rswzrxlgzeejmieq3n5aal.parquet',
        'walk':'https://uchicago.box.com/shared/static/lrxeqclmpkflibg9c7sphmun4kny9xsb.parquet'
    },
    'zip': {
        'car':'https://uchicago.box.com/shared/static/swggh8jxj59c7vpxzx1emt7jnd083rmh.parquet',
        'bike':'https://uchicago.box.com/shared/static/7yzgf1gx3k3sacntjqber6l40m0d5aqw.parquet', 
        'walk':'https://uchicago.box.com/shared/static/b3vuqijqys24z146u78dsemn0wvu8i5m.parquet',  
    }
}

geographies = {
    'tract': 'https://uchicago.box.com/shared/static/kfoid6fzlbpyfecmwpe9u16cl5n89ru0.zip',
    'zip':'https://uchicago.box.com/shared/static/270ca6syxcg3dlvohhnt3ap92m4t7cxc.zip'
}
pop_data = {
    'tract':'https://uchicago.box.com/shared/static/z6xm6tre935xbc06gg4ukzgyicro26cw.csv',
    'zip': 'https://uchicago.box.com/shared/static/njjpskiuj7amcztrxjws2jfwqlv66t49.csv'
}
sample_point_data = {
    'FQHC': 'https://uchicago.box.com/shared/static/uylcq23g5z8jhvmp7cnofr074j4hwj6e.csv',
    'pharmacies': 'https://uchicago.box.com/shared/static/njjpskiuj7amcztrxjws2jfwqlv66t49.csv',
    'opioid_treatment_facilities': 'https://raw.githubusercontent.com/GeoDaCenter/opioid-policy-scan/master/data_raw/Opioid_Treatment_Directory_Geocoded.csv',
    'moud_full': 'https://raw.githubusercontent.com/GeoDaCenter/opioid-policy-scan/master/data_final/moud/us-wide-moudsCleaned_geocoded.csv'
}

geoid_cols = {
    "tract":"GEOID",
    "zip": "GEOID10"
}

Specify your preferred geographic unit and transit mode below. Or do your own thing!

In [8]:
GEOGRAPHIC_UNIT = 'zip' # 'tract' or 'zip'
TRANSIT_MODE = 'walk' # 'car' or 'bike' or 'walk'

TRANSIT_MATRIX = pd.read_parquet(matrices[GEOGRAPHIC_UNIT][TRANSIT_MODE])

In [9]:
TRANSIT_MATRIX.head()

,origin,destination,minutes
0,99660,99660,0.00
1,99685,99685,0.00
2,99685,99692,96.33
3,99692,99685,96.33
4,99692,99692,0.00


In [10]:
GEOGRAPHIES = gpd.read_file(geographies[GEOGRAPHIC_UNIT]).to_crs('EPSG:4326')
GEOGRAPHIES['FIPS'] = GEOGRAPHIES[geoid_cols[GEOGRAPHIC_UNIT]].astype('int64')

In [11]:
# update your destination of interest here (i.e. point locations csv) with lat/long coordinates
DESTINATIONS = dfToGdf(pd.read_csv(sample_point_data['moud_full']), 'xcoord', 'ycoord') 

In [12]:
# Update your destinations to a subset or category; i.e. category of MOUD
DESTINATIONS = DESTINATIONS[DESTINATIONS.category == "methadone"]
#bup_providers = sample_point_data.query(`category == 'bup'`)

In [ ]:
# If you're trying to set up some gravity models or other access score calculations,  you'll probably need population data. 
# We've got you covered. 
# POPULATION = pd.read_csv(pop_data[GEOGRAPHIC_UNIT])[['FIPS','Total Population']].iloc[1:]
# POPULATION['GEOID'] = POPULATION['FIPS'].astype('int64')
# GEOGRAPHIES = GEOGRAPHIES.merge(POPULATION[['FIPS','Total Population']], how="left", on="FIPS")

### Spatial Join
Spatially joining points and polygons is easy. We're using the `intersects` geometric predicate here, simply meaning that if a point intersects a polygon, those two become joined or associated. 

This means that we are able to see which polygon from our geographies each destination is in, and from the travel matrix, we'll know how far it is (roughly) from the other georaphies.

In [13]:
merged_destinations = gpd.sjoin(DESTINATIONS, GEOGRAPHIES[['FIPS', 'geometry']], how='inner', op='intersects')
merged_destinations.head()

/Users/susan/opt/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3364: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


,name1,name2,street1,street2,city,state,zip,category,countyName,source,geometry,index_right,FIPS
0,Oklahoma Treatment Services LLC,Bartlesville Rightway Medical,610 West Hensley Boulevard,NaN,Bartlesville,OK,74003,methadone,Washington,SAMHSA,POINT (-95.98404 36.75355),549,74003
1,Quapaw Counseling Services,NaN,58150 East 66 Road,NaN,Miami,OK,74354,methadone,Ottawa,SAMHSA,POINT (-94.83930 36.91948),26718,74354
2,Keetoowah Cherokee Trt Servs,NaN,2727 East Admiral Place,NaN,Tulsa,OK,74110,methadone,Tulsa,SAMHSA,POINT (-95.95045 36.16111),26906,74110
3,BHG Joplin Treatment Center,NaN,2919 East 4th Street,NaN,Joplin,MO,64801,methadone,Jasper,SAMHSA,POINT (-94.47820 37.08740),22907,64801
4,Tulsa Rightway Medical,NaN,3445 South Sheridan Road East,NaN,Tulsa,OK,74145,methadone,Tulsa,SAMHSA,POINT (-95.90032 36.11242),29323,74145


In [14]:
# Check dimensions
merged_destinations.shape

(1442, 13)

### Moins Est Plus
Less is more, let's just snag the columns we need. We'll need to join this data again to the travel matrix, so the second line gets everyone speaking the same language.

In [15]:
## Pull out the simplified columns we need for the analysis
destinations_simplified = merged_destinations[['index_right','FIPS']]
## Type correctly to merge
## Ignore that warning this is fine
#destinations_simplified.loc['FIPS'] = destinations_simplified['FIPS'].astype('int64') 
destinations_simplified.head()

,index_right,FIPS
0,549,74003
1,26718,74354
2,26906,74110
3,22907,64801
4,29323,74145


In [16]:
# Check dimensions
destinations_simplified.shape

(1442, 2)

### The Big Join
Currently, we have destinations associated with our origin geographies (if you're using the default data, census tracts and health clinics), but we need to bring it all together by joining the origins and destinations to the travel matrix. Below, we'll join the travel matrix to the destinations:

In [17]:
## Merge onto the transit matrix, giving us the distance from each origin to each destination
merge_transit_matrix = TRANSIT_MATRIX.merge(destinations_simplified, left_on="destination", right_on="FIPS")
merge_transit_matrix.head()

,origin,destination,minutes,index_right,FIPS
0,35580,35504,335.25,5788,35504
1,35541,35504,421.91,5788,35504
2,35553,35504,544.60,5788,35504
3,35550,35504,296.12,5788,35504
4,35579,35504,397.90,5788,35504


### Analysis Time
Let's get down to business. To begin, let's declare some variables that will help us a bit later. To start, we can define what our origin column (by default, creatively, `origin`), the destination ID column that came from the destinations data, the travel cost column, and the treshold for travel time. 

In [18]:
origin_col = 'origin'
destination_id_col = 'index_right'
travel_cost_col = 'minutes'
travel_threshold60 = 60
travel_threshold30 = 30

### Data Cleanup
We have some weird -1000 values. Let's fix them and replace them with 999, the default null value of this travel matrix.

In [19]:
## clean up this weird bug
## then merge the data
travel_costs = merge_transit_matrix.sort_values(travel_cost_col, ascending=True)
travel_costs.minutes = travel_costs.minutes.replace(-1000, 999)
travel_costs.origin = travel_costs.origin.astype('int64')
travel_costs.head()

,origin,destination,minutes,index_right,FIPS
39167,23696,23072,999.0,21422,23072
677,85707,85711,999.0,31909,85711
7739,32212,32207,999.0,12484,32207
567,85708,85710,999.0,29359,85710
674,85708,85711,999.0,31909,85711


### Nearest location
To get the nearest location, sort the values by lowest cost then filter for  the first appearance of each origin ID. This means we'll get the first time that origin shows up, sorted by the lowest travel cost. 

We'll use pandas `.duplicated()` function with the not (`~`) operator before it.

In [20]:
time_to_nearest = travel_costs[~travel_costs.origin.duplicated()][[origin_col, travel_cost_col]]
time_to_nearest.head()

,origin,minutes
39167,23696,999.0
677,85707,999.0
7739,32212,999.0
567,85708,999.0
39154,23025,999.0


### Count in Threshold 
For getting the count of destinations within a given threshold (by default, 30 minutes), we can chain a couple functions here from pandas. 

First, we'll filter the `travel_costs` dataframe for costs that are less than or equal to the threshold. Then group by the origin column, giving us sets of rows that share the same origin ID, and then count those columns, giving us the number of rows for each origin ID with a travel cost under our treshold.

We'll re-label some columns for easy reference.

In [21]:
## For count, we simply filter for the cost under a given threshold (60 minutes)
## Then group by by and count the results
count_within_threshold60 = travel_costs[travel_costs[travel_cost_col] <= travel_threshold60] \
  .groupby(origin_col).count() \
  .reset_index()[[origin_col, travel_cost_col]]
count_within_threshold60.columns = [origin_col, f"count within {travel_threshold60}"]

In [22]:
count_within_threshold60.head()

,origin,count within 60
0,1013,1
1,1020,1
2,1040,2
3,1060,2
4,1062,2


In [23]:
## For count, we simple filter for the cost under a given threshold (60 minutes)
## Then group by by and count the results
count_within_threshold30 = travel_costs[travel_costs[travel_cost_col] <= travel_threshold30] \
  .groupby(origin_col).count() \
  .reset_index()[[origin_col, travel_cost_col]]

count_within_threshold30.columns = [origin_col, f"count within {travel_threshold30}"]

In [24]:
count_within_threshold30.head()

,origin,count within 30
0,1013,1
1,1040,2
2,1060,2
3,1063,2
4,1085,1


In [25]:
# Merge threshold 60 and 30 minutes
count_within_threshold = count_within_threshold60.merge(count_within_threshold30, on=origin_col, how="outer")
count_within_threshold.head()

,origin,count within 60,count within 30
0,1013,1,1.0
1,1020,1,NaN
2,1040,2,2.0
3,1060,2,2.0
4,1062,2,NaN


### Merge Results
Now, we can merge our two findings into an easy, breezy, beautiful dataframe.

In [26]:
merged_metrics = count_within_threshold.merge(time_to_nearest, on=origin_col, how="outer")
merged_metrics.head()

,origin,count within 60,count within 30,minutes
0,1013,1.0,1.0,0.00
1,1020,1.0,NaN,43.59
2,1040,2.0,2.0,0.00
3,1060,2.0,2.0,0.00
4,1062,2.0,NaN,56.20


### Cleanup
One last edge case to handle here: It is possible some origins are not within 30 minutes of a destination, meaning some of the data will be null. Or, we might have lost some origins from the full geographies dataset.

While not the end of the world, we can clean this up here before shipping of results to our (soon to be disgruntled) data scientist colleagues. 

The below finds the missing origin IDs and fills them in, giving us the revered `findings` dataframe. 

🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉

In [27]:
## To clean up any missing data, we can check back with our origin list
analyzed_origins = list(merged_metrics[origin_col])
missing_origins = [o for o in GEOGRAPHIES.FIPS if o not in analyzed_origins]

## Then, fill the missing data
missing_data = []
for o in missing_origins:
    missing_entry = {}
    missing_entry[origin_col] = o
    missing_entry[f"count within {travel_threshold60}"]=0
    missing_entry[f"count within {travel_threshold30}"]=0
    missing_entry[travel_cost_col]=None
    missing_data.append(missing_entry)
missing_df = pd.DataFrame(missing_data)

## and concatenate results
findings = pd.concat([merged_metrics, missing_df])
# Fill any null values with 0 for count within
findings['count within 30'] = findings['count within 30'].fillna(0).astype(int)
findings['count within 60'] = findings['count within 60'].fillna(0).astype(int)
# Replace error value "999" in matrices with blanks
findings['minutes'] = findings['minutes'].replace(999.0, None)
findings.head()

,origin,count within 60,count within 30,minutes
0,1013,1,1,0.00
1,1020,1,0,43.59
2,1040,2,2,0.00
3,1060,2,2,0.00
4,1062,2,0,56.20


### What's Next?
Well, now you could take this data and export it as a CSV, or join it back to the geographies and visualize it, or try running this analysis with some different data. Ball is in your court, you got this!


In [28]:
# Check dimensions for all zip codes; should be ~33,000 
findings.shape

(33144, 4)

In [29]:
# Export to csv
findings.to_csv('meth_zip_walkAccess.csv', index = False)
